In [8]:
import os
import pathlib
from pprint import pprint

In [3]:
from dotenv import load_dotenv
load_dotenv(".env")
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment":os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

In [2]:
res_folder_path='./results'
if not os.path.exists(res_folder_path):
    os.makedirs(res_folder_path)
res_file=res_folder_path + '/results.csv'
with open(res_file, 'w') as f:
    f.write('model, prompt, completion\n')

In [ ]:
# generate yml file from prompt file
# api function file folder
from prompteval.prompteval import get_language_from_extension, generateApiSpec, remove_code_block_format
api_func_folder = pathlib.Path('./apisrc')
res_folder_path = pathlib.Path('./results')
for api_file in api_func_folder.glob('*'):
    with open(api_file, 'r', encoding='utf-8') as f:
        api_func = f.read()
    file_extension = api_file.suffix[1:]
    language = get_language_from_extension(file_extension)
    response = generateApiSpec(api_func, language)
    filename = f"{api_file.stem}_response.yml"
    output_file = res_folder_path / filename
    with open(output_file, "w", encoding="utf-8") as out_file:
        out_file.write(remove_code_block_format(response["response"]))

In [ ]:
# spectral analysis for results
import json
from prompteval.analyse import any_spec_res, text_to_json_objects, spectral_analyse
ruleset_url = "https://raw.githubusercontent.com/Azure/APICenter-Analyzer/preview/resources/rulesets/oas.yaml"
res_folder_path = pathlib.Path('./results')
results = []
calcs = []
for res_file in res_folder_path.glob("*"):
    if res_file.suffix == ".yml":
      out_file = os.path.basename(res_file.stem + "_output.json")
      spectral_analyse(os.path.basename(res_file), ruleset_url, res_folder_path, os.path.basename(out_file))
      res, calc = any_spec_res(text_to_json_objects(res_folder_path, out_file))
      results.append(res)
      calcs.append(calc)

with open(pathlib.Path(res_folder_path, 'output.jsonl'), 'w') as jsonl_file:
# 遍历数据中的每个对象
    for item in calcs:
    # 将每个对象转换回JSON字符串，并写入文件，每个对象占一行
      jsonl_file.write(json.dumps(item) + '\n')


In [16]:
pprint(results)

[[{'code': 'info-contact',
   'message': 'Info object must have "contact" object.',
   'severity': 1},
  {'code': 'operation-description',
   'message': 'Operation "description" must be present and non-empty string.',
   'severity': 1},
  {'code': 'operation-tag-defined',
   'message': 'Operation tags must be defined in global tags.',
   'severity': 1}],
 [{'code': 'info-contact',
   'message': 'Info object must have "contact" object.',
   'severity': 1},
  {'code': 'operation-operationId',
   'message': 'Operation must have "operationId".',
   'severity': 1},
  {'code': 'operation-tags',
   'message': 'Operation must have non-empty "tags" array.',
   'severity': 1}],
 [{'code': 'oas3-api-servers',
   'message': 'OpenAPI "servers" must be present and non-empty array.',
   'severity': 1},
  {'code': 'info-contact',
   'message': 'Info object must have "contact" object.',
   'severity': 1},
  {'code': 'info-description',
   'message': 'Info "description" must be present and non-empty str